In [ ]:
# !pip install import-ipynb
import import_ipynb
from environment import TicTacToe3D

import random
import numpy as np

In [ ]:
def calculateAllWin():
    directions = [
        (1, 0, 0), (0, 1, 0), (0, 0, 1),  # Linear along each dimension
        (1, 1, 0), (1, -1, 0), (1, 0, 1), (1, 0, -1), (0, 1, 1), (0, 1, -1),  # Planar diagonals
        (1, 1, 1), (1, 1, -1), (1, -1, 1), (-1, 1, 1)  # Space diagonals
    ]
    
    all_wins = []
    board_size = 4
    win_length = 4

    # Iterate over each cell as a possible start of a winning line
    for x in range(board_size):
        for y in range(board_size):
            for z in range(board_size):
                for dx, dy, dz in directions:
                    # Check if a line can be drawn from (x, y, z) in (dx, dy, dz) direction
                    end_x = x + (win_length - 1) * dx
                    end_y = y + (win_length - 1) * dy
                    end_z = z + (win_length - 1) * dz
                    # Ensure the line stays within bounds of the board
                    if 0 <= end_x < board_size and 0 <= end_y < board_size and 0 <= end_z < board_size:
                        # Create a mask for this winning line
                        win_mask = np.zeros((board_size, board_size, board_size), dtype=int)
                        for i in range(win_length):
                            win_mask[x + i * dx, y + i * dy, z + i * dz] = 1
                        all_wins.append(win_mask)
    
    return all_wins

In [ ]:
class RandomAgent:
    def __init__(self):
        self.all_wins = calculateAllWin()

    def findBestMove(self, board, possible_move, player):
        game = TicTacToe3D().loadState(board)

        if not len(possible_move) == 0:
            return None
        
        initial_blocks = set()
        for win in self.all_wins:
            line = board * win
            if np.count_nonzero(line == -player) == 3 and np.count_nonzero(line == player) == 1:
                initial_blocks.add(str(win.nonzero()))  # Store win patterns that are already blocking moves
        
        bestMove = random.choice(possible_move)

        for i, j in possible_move:
            game.move(i, j, player)
            new_board = game.board.copy()

            for win in self.all_wins:
                line = new_board.copy() * win.copy()
                if np.count_nonzero(line == player) == 4:
                    return (i, j)
                elif np.count_nonzero(line == -player) == 3 and np.count_nonzero(line == player) == 1 and str(win.nonzero()) not in initial_blocks:
                    return (i, j)

            game.loadState(board)

        return bestMove